In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Yellowknife,CA,2022-04-05 23:51:12,62.4560,-114.3525,31.41,74,100,10.36
1,1,Severomuysk,RU,2022-04-05 23:51:12,56.1481,113.4267,4.59,79,100,2.24
2,2,Puerto Colombia,CO,2022-04-05 23:51:13,10.9878,-74.9547,82.49,85,99,18.77
3,3,Mpraeso,GH,2022-04-05 23:51:13,6.5932,-0.7346,74.19,91,94,3.38
4,4,Hilo,US,2022-04-05 23:51:13,19.7297,-155.0900,76.73,78,100,8.05


In [3]:
#Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Puerto Colombia,CO,2022-04-05 23:51:13,10.9878,-74.9547,82.49,85,99,18.77
4,4,Hilo,US,2022-04-05 23:51:13,19.7297,-155.0900,76.73,78,100,8.05
11,11,Rikitea,PF,2022-04-05 23:51:15,-23.1203,-134.9692,78.66,68,43,17.65
27,27,Kavieng,PG,2022-04-05 23:51:21,-2.5744,150.7967,80.49,81,100,13.96
31,31,Funtua,NG,2022-04-05 23:51:21,11.5233,7.3081,80.85,12,100,5.59
32,32,Jaciara,BR,2022-04-05 23:51:22,-15.9653,-54.9683,77.25,71,94,0.22
39,39,Vaini,TO,2022-04-05 23:51:24,-21.2000,-175.2000,82.56,78,75,3.44
43,43,Saint-Philippe,RE,2022-04-05 23:52:26,-21.3585,55.7679,76.59,87,81,9.31
46,46,Atuona,PF,2022-04-05 23:51:08,-9.8000,-139.0333,79.34,71,1,22.44
47,47,Namatanai,PG,2022-04-05 23:52:27,-3.6667,152.4333,77.76,91,100,8.48


In [9]:
preferred_cities_df.count()

City_ID       157
City          157
Country       157
Date          157
Lat           157
Lng           157
Max Temp      157
Humidity      157
Cloudiness    157
Wind Speed    157
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Puerto Colombia,CO,82.49,10.9878,-74.9547,
4,Hilo,US,76.73,19.7297,-155.0900,
11,Rikitea,PF,78.66,-23.1203,-134.9692,
27,Kavieng,PG,80.49,-2.5744,150.7967,
31,Funtua,NG,80.85,11.5233,7.3081,
32,Jaciara,BR,77.25,-15.9653,-54.9683,
39,Vaini,TO,82.56,-21.2000,-175.2000,
43,Saint-Philippe,RE,76.59,-21.3585,55.7679,
46,Atuona,PF,79.34,-9.8000,-139.0333,
47,Namatanai,PG,77.76,-3.6667,152.4333,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Puerto Colombia,CO,82.49,10.9878,-74.9547,Hotel El Emigrante
4,Hilo,US,76.73,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Rikitea,PF,78.66,-23.1203,-134.9692,People ThankYou
27,Kavieng,PG,80.49,-2.5744,150.7967,Nusa Island Retreat
31,Funtua,NG,80.85,11.5233,7.3081,Sani Jikan Mallam Hotel
...,...,...,...,...,...,...
562,Pitiquito,MX,87.60,30.7000,-112.0833,La Víspera
568,Monrovia,LR,79.03,6.3005,-10.7969,Sunset Inn
569,Wagar,SD,82.44,16.1525,36.2032,
571,Kloulklubed,PW,82.90,7.0419,134.2556,Storyboard Beach Resort


In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))